In [24]:
from py2neo import Graph
graph = Graph("bolt: // localhost:7474", auth=("neo4j", "12345678"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
from py2neo import Node
# 创建 角色 节点 郭靖，拥有属性 姓名-郭靖，性别-男
guojing = Node('角色',name='郭靖',sex='男')
# 创建 角色 节点 黄蓉，拥有属性 姓名-黄蓉，性别-女
huangrong = Node('角色',name='黄蓉',sex='女')
# 创建 作品 节点 射雕英雄传，拥有属性 名称-射雕英雄传，作者-金庸
shediao = Node('作品',name='射雕英雄传',author='金庸')
# 打印结果
guojing,huangrong,shediao


(Node('角色', name='郭靖', sex='男'),
 Node('角色', name='黄蓉', sex='女'),
 Node('作品', author='金庸', name='射雕英雄传'))

In [14]:
# 获取 key 对应的属性
guojing['name']
# 设置 key 对应属性的 value，如果 key 不存在就创建
huangrong['born'] = '桃花岛'
huangrong
# 删除某个属性
del huangrong['born']
huangrong
# 返回节点中 property 的个数
len(huangrong)


2

In [15]:
# 在 Neo4j 数据库中创建 郭靖 节点
graph.create(guojing)
# 在 Neo4j 数据库中创建 黄蓉 节点
graph.create(huangrong)
# 在 Neo4j 数据库中创建 射雕英雄传 节点
graph.create(shediao)
# 打印结果
guojing,huangrong,shediao


(Node('角色', name='郭靖', sex='男'),
 Node('角色', name='黄蓉', sex='女'),
 Node('作品', author='金庸', name='射雕英雄传'))

In [16]:
# 反复执行此命令来创建 郭靖 节点
graph.create(guojing)


In [17]:
# 重新 guojing 变量赋值
guojing = Node('角色',name='郭靖',sex='男')
guojing

Node('角色', name='郭靖', sex='男')

In [18]:
from py2neo import Relationship
# 创建郭靖与黄蓉的配偶关系
spouse1 = Relationship(guojing,'配偶',huangrong)
# 创建黄蓉与郭靖的配偶关系
spouse2 = Relationship(huangrong,'配偶',guojing)
# 创建郭靖与射雕英雄传的所在作品关系
inbook1 = Relationship(guojing,'所在作品',shediao)
# 创建黄蓉与射雕英雄传的所在作品关系
inbook2 = Relationship(huangrong,'所在作品',shediao)
# 打印结果
spouse1,spouse2,inbook1,inbook2


(配偶(Node('角色', name='郭靖', sex='男'), Node('角色', name='黄蓉', sex='女')),
 配偶(Node('角色', name='黄蓉', sex='女'), Node('角色', name='郭靖', sex='男')),
 所在作品(Node('角色', name='郭靖', sex='男'), Node('作品', author='金庸', name='射雕英雄传')),
 所在作品(Node('角色', name='黄蓉', sex='女'), Node('作品', author='金庸', name='射雕英雄传')))

In [26]:
# 设置 key 对应属性的 value
inbook1['role'] = '男主角'
inbook1
# 删除某个属性
del inbook1['role']
inbook1


所在作品(Node('角色', name='郭靖', sex='男'), Node('作品', author='金庸', name='射雕英雄传'), role='男主角')

所在作品(Node('角色', name='郭靖', sex='男'), Node('作品', author='金庸', name='射雕英雄传'))

In [20]:
# 创建郭靖与黄蓉的配偶关系
graph.create(spouse1)
# 创建黄蓉与郭靖的配偶关系
graph.create(spouse2)
# 创建郭靖与射雕英雄传的所在作品关系
graph.create(inbook1)
# 创建黄蓉与射雕英雄传的所在作品关系
graph.create(inbook2)


In [25]:
from py2neo import NodeMatcher
# 初始化一个 matcher 实例
matcher = NodeMatcher(graph)
# 用 match 方法查找 角色 中 name 为郭靖的节点，返回一个 NodeMatch 对象
result = matcher.match("角色", name="郭靖")
# first 方法返回查询结果的第一个
result.first()
# 通过 list 来把所有结果显示出来
list(result)


Node('角色', name='郭靖', sex='男')

[Node('角色', name='郭靖', sex='男'), Node('角色', name='郭靖', sex='男')]

In [27]:
# cypher 语句，对已存在的 郭靖 节点，创建节点和关系 (郭靖)-[父]->(郭啸天)
query = "match (n) where n.name = '郭靖' create (n)-[:父]->(:角色{name:'郭啸天'})"
# 执行 cypher 语句
graph.run(query)


(No data)

In [28]:
# cypher 语句，查询并返回所有的节点
query = "match (n) return n"
# 执行 cypher 语句，获得返回结果
cursors = graph.run(query)
# 通过遍历的方式取出所有结果
for i in cursors:
    print(i)


Node('角色', name='郭靖', sex='男')
Node('角色', name='黄蓉', sex='女')
Node('作品', author='金庸', name='射雕英雄传')
Node('角色', name='郭靖', sex='男')
Node('角色', name='郭啸天')
Node('角色', name='郭啸天')


In [30]:


# 定义人物关系三元组列表
person_list = [('杨过', '配偶', '小龙女'), ('杨过', '父', '杨康'), ('杨康', '父', '杨铁心'), ('小龙女', '配偶', '杨过')]

# 创建人物节点和关系
for triple in person_list:
    person1 = Node("Person", name=triple[0])
    person2 = Node("Person", name=triple[2])
    relationship = Relationship(person1, triple[1], person2)
    graph.create(person1 | person2 | relationship)
